### Simulation at the same operating conditions as real data

To compare a model with operation data, we need to run the simulation at the same operating conditions to avoid a bias.

For that, we will:
- create a model
- load the characteristics
- load the operation data
- run a simulation using those data

In [ ]:
from cpu.systems import CPUSystem

# create a new CPU model/system
cpu = CPUSystem("cpu")

# load the characteristics
with open("data/cpu_ref.json") as f:
    cpu.load(f)

In [ ]:
import numpy as np
import pandas as pd

from cosapp.drivers import RungeKutta, NonLinearSolver
from cosapp.drivers.time.scenario import Interpolator
from cosapp.recorders import DataFrameRecorder

time_driver = cpu.add_driver(RungeKutta(order=3, history = True))
solver = time_driver.add_child(NonLinearSolver('solver', max_iter=10, factor=1.0))

time_driver.time_interval = (0, 30)
time_driver.dt = 1.

# load the operation data
data = pd.read_csv("data/cpu_hot_day_intensive_use.csv")

# define a simulation scenario using the operating conditions
time_driver.set_scenario(
    init = {'T_cpu': 10.},
    values = {
        "fan.T_air": Interpolator(np.stack([data.index, data["fan.T_air"]], axis=1)),
        "cpu.usage": Interpolator(np.stack([data.index, data["cpu.usage"]], axis=1)),
    }
)

rec = time_driver.add_recorder(DataFrameRecorder(includes=['*'], hold=False), period=1.)

cpu.run_drivers()
rec.data.to_csv("data/simulation_at_operating_conditions.csv")

In [ ]:
from cpu.utils.plot import plot_recorders

plot_recorders({"operation": pd.read_csv("data/cpu_hot_day_intensive_use.csv"),
                "simulation": pd.read_csv("data/simulation_at_operating_conditions.csv"),
                }, 
               [[("time", "fan.T_air"), ("time","cpu.usage")], 
                [("time", "fan.tension"), ("time", "T_cpu")]],
               width=800, height=600)